## Configs

In [2]:
import os 
import pandas as pd 
# private key to access the storage account 
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='./keyfile.json' # Specify the gcs authentification file path 


In [3]:
# Install packages
!pip install mlflow
!pip install google-cloud 
!pip install google-cloud-storage
!pip install xgboost

     |████████████████████████████████| 14.4 MB 14.6 MB/s eta 0:00:01   |██████████████████▊             | 8.4 MB 5.9 MB/s eta 0:00:02
     |████████████████████████████████| 249 kB 12.2 MB/s eta 0:00:01
  Using cached gunicorn-20.1.0-py3-none-any.whl (79 kB)
     |████████████████████████████████| 16.9 MB 147 kB/s  eta 0:00:01    |█▏                              | 593 kB 14.9 MB/s eta 0:00:02     |███████████▉                    | 6.2 MB 14.9 MB/s eta 0:00:01     |███████████████████████████████▊| 16.8 MB 14.9 MB/s eta 0:00:01
  Using cached Flask-2.0.1-py3-none-any.whl (94 kB)
  Using cached docker-5.0.0-py2.py3-none-any.whl (146 kB)
  Using cached alembic-1.4.1.tar.gz (1.1 MB)
  Using cached querystring_parser-1.2.4-py2.py3-none-any.whl (7.9 kB)
  Using cached click-8.0.1-py3-none-any.whl (97 kB)
  Using cached protobuf-3.17.3-cp37-cp37m-macosx_10_9_x86_64.whl (1.0 MB)
  Using cached databricks-cli-0.14.3.tar.gz (54 kB)
  Using cached prometheus_flask_exporter-0.18.2.tar.gz (22 kB)


     |████████████████████████████████| 152 kB 9.6 MB/s eta 0:00:01
     |████████████████████████████████| 75 kB 7.9 MB/s  eta 0:00:01
  Using cached cachetools-4.2.2-py3-none-any.whl (11 kB)
     |████████████████████████████████| 155 kB 16.0 MB/s eta 0:00:01
     |████████████████████████████████| 93 kB 2.1 MB/s  eta 0:00:01
     |████████████████████████████████| 198 kB 20.1 MB/s eta 0:00:01
  Using cached google_crc32c-1.1.2-cp37-cp37m-macosx_10_14_x86_64.whl (27 kB)
     |████████████████████████████████| 176 kB 18.4 MB/s eta 0:00:01
  Using cached pycparser-2.20-py2.py3-none-any.whl (112 kB)
     |████████████████████████████████| 77 kB 8.6 MB/s  eta 0:00:01
  Using cached xgboost-1.4.2-py3-none-macosx_10_14_x86_64.macosx_10_15_x86_64.macosx_11_0_x86_64.whl (1.2 MB)
     |████████████████████████████████| 31.9 MB 6.3 MB/s eta 0:00:01    |▎                               | 235 kB 5.1 MB/s eta 0:00:07     |█████████████████▏              | 17.1 MB 7.5 MB/s eta 0:00:02


In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
import xgboost as xgb
import mlflow
import mlflow.xgboost

mlflow.set_tracking_uri('http://loadbalancerip') # Change to Mlflow URI  (loadbalancer ip)
print("MLflow Version:", mlflow.__version__)
print("MLflow Tracking URI:", mlflow.get_tracking_uri())
print("XGBoost version:",xgb.__version__)
client = mlflow.tracking.MlflowClient()



MLflow Version: 1.14.1
MLflow Tracking URI: http://loadbalancerip
XGBoost version: 0.90


## Train and register a model

In [1]:
def build_data(data_path):
    data = pd.read_csv(data_path)
    train, test = train_test_split(data, test_size=0.3, random_state=1)

    # The predicted column is "quality" which is a scalar from [3, 9]
    X_train, X_test = train.drop(["quality"], axis=1), test.drop(["quality"], axis=1)
    y_train, y_test = train["quality"], test["quality"]

    return X_train, X_test, y_train, y_test 


In [5]:
def train(data_path, max_depth, min_child_weight, estimators, model_name):
    X_train, X_test, y_train, y_test = build_data(data_path)
    with mlflow.start_run() as run:
        # Start mlflow session
        run_id = run.info.run_uuid
        experiment_id = run.info.experiment_id
        print("MLflow:")
        print("  run_id:", run_id)
        print("  experiment_id:", experiment_id)
        print("  experiment_name:", client.get_experiment(experiment_id).name)

        # MLflow params
        print("Parameters:")
        print("  max_depth:", max_depth)
        print("  min_child_weight:", min_child_weight)
        print("  estimators:", estimators)
        
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_param("min_child_weight", min_child_weight)
        mlflow.log_param("estimators", estimators)

        # Create and fit model
        model = xgb.XGBRegressor(
                 max_depth=max_depth,
                 min_child_weight=min_child_weight,
                 random_state=42) 
        model.fit(X_train, y_train)
        
        # MLflow metrics
        predictions = model.predict(X_test)
        print("predictions:",predictions)
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        mae = mean_absolute_error(y_test, predictions)
        r2 = r2_score(y_test, predictions)
        
        print("Metrics:")
        print("  rmse:", rmse)
        print("  mae:", mae)
        print("  r2:", r2)
        
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        # Log model
        mlflow.xgboost.log_model(model, "xgboost-model", registered_model_name = model_name)


In [6]:
data_path = './wine-quality-white.csv'
experiment_name = 'test_xgboost'
model_name = 'xgb_0'
max_depth = 10
min_child_weight = 1
estimators = 100
train(data_path, max_depth , min_child_weight, estimators, model_name)


MLflow:
  run_id: b66effdfef4e477ebacb3859a32b424a
  experiment_id: 0
  experiment_name: Exp1
Parameters:
  max_depth: 10
  min_child_weight: 1
  estimators: 100
predictions: [5.930445  6.9174986 6.943309  ... 6.4364724 6.987052  5.6387153]
Metrics:
  rmse: 0.6605505923590334
  mae: 0.44089301443424356
  r2: 0.44297057516776106


Successfully registered model 'xgb_0'.
2021/07/16 11:28:41 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: xgb_0, version 1
Created version '1' of model 'xgb_0'.


## Load the model from mlflow and make predictions

In [7]:
# Predict on a Pandas DataFrame

import pandas as pd

test_samples =pd.read_csv(data_path).head(5).drop(columns=['quality'])
loaded_model = mlflow.pyfunc.load_model("runs:/a0928931dff54a829b881be2a3e41d00/xgboost-model")
loaded_model.predict(test_samples)

array([6.0004535, 5.994977 , 5.629604 , 5.9799094, 5.9799094],
      dtype=float32)